## AB TESTING

##### Bir giriş ekranının mavi renk ağırlıklı mı kırmızı renk ağırlıklı mı olması gerektiği yönünde bir araştırma yapılmıştır.
##### Kullanıcıların günlük ekrana gelişlerdeki tıklama, satın alma ve gelir miktarlarını içeren veriler toplanmıştır.

### Import

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats import proportion as pr
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import warnings
import statsmodels.stats.api as sms

# AB TESTING
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, pearsonr, spearmanr, kendalltau, \
    f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

In [ ]:
warnings.filterwarnings ("ignore")
pd.set_option ('display.float_format', lambda x: '%.6f' % x)
pd.set_option ('display.width', None)
pd.set_option ('display.max_columns', None)

### DataSet Load

In [ ]:
df = pd.read_excel("ab_data.xlsx")
df

,day,color,color_count,click_count,click_rate,purchase_count,purchase_rate,avg_revenue
0,1,blue,25,9,0.360000,6,0.240000,74.333333
1,1,red,25,12,0.480000,11,0.440000,74.818182
2,2,blue,18,9,0.500000,5,0.277778,75.400000
3,2,red,32,21,0.656250,11,0.343750,73.181818
4,3,blue,20,11,0.550000,7,0.350000,73.571429
...,...,...,...,...,...,...,...,...
115,58,red,20,9,0.450000,5,0.250000,71.000000
116,59,blue,27,12,0.444444,6,0.222222,66.166667
117,59,red,23,10,0.434783,6,0.260870,78.500000
118,60,blue,26,14,0.538462,9,0.346154,76.666667


### Data PreProcessing

In [ ]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head())
    print("##################### Tail #####################")
    print(dataframe.tail())
    print("##################### NA #####################")
    print(dataframe.isnull().sum())

check_df(df)

##################### Shape #####################
(120, 8)
##################### Types #####################
day                 int64
color              object
color_count         int64
click_count         int64
click_rate        float64
purchase_count      int64
purchase_rate     float64
avg_revenue       float64
dtype: object
##################### Head #####################
   day color  color_count  click_count  click_rate  purchase_count  purchase_rate  avg_revenue
0    1  blue           25            9    0.360000               6       0.240000    74.333333
1    1   red           25           12    0.480000              11       0.440000    74.818182
2    2  blue           18            9    0.500000               5       0.277778    75.400000
3    2   red           32           21    0.656250              11       0.343750    73.181818
4    3  blue           20           11    0.550000               7       0.350000    73.571429
##################### Tail #####################
 

In [ ]:
df. shape

(120, 8)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   day             120 non-null    int64  
 1   color           120 non-null    object 
 2   color_count     120 non-null    int64  
 3   click_count     120 non-null    int64  
 4   click_rate      120 non-null    float64
 5   purchase_count  120 non-null    int64  
 6   purchase_rate   120 non-null    float64
 7   avg_revenue     120 non-null    float64
dtypes: float64(3), int64(4), object(1)
memory usage: 7.6+ KB


In [ ]:
# Blue ve Red gruplarını ayırma
blue_data = df[df['color'] == 'blue']
red_data = df[df['color'] == 'red']

In [ ]:
blue_data

,day,color,color_count,click_count,click_rate,purchase_count,purchase_rate,avg_revenue
0,1,blue,25,9,0.360000,6,0.240000,74.333333
2,2,blue,18,9,0.500000,5,0.277778,75.400000
4,3,blue,20,11,0.550000,7,0.350000,73.571429
6,4,blue,28,18,0.642857,15,0.535714,74.666667
8,5,blue,33,21,0.636364,10,0.303030,70.200000
10,6,blue,30,18,0.600000,10,0.333333,78.000000
12,7,blue,30,23,0.766667,13,0.433333,76.923077
14,8,blue,27,16,0.592593,8,0.296296,75.125000
16,9,blue,25,11,0.440000,6,0.240000,65.166667
18,10,blue,20,11,0.550000,7,0.350000,69.142857


In [ ]:
red_data

,day,color,color_count,click_count,click_rate,purchase_count,purchase_rate,avg_revenue
1,1,red,25,12,0.480000,11,0.440000,74.818182
3,2,red,32,21,0.656250,11,0.343750,73.181818
5,3,red,30,17,0.566667,7,0.233333,75.428571
7,4,red,22,14,0.636364,6,0.272727,71.833333
9,5,red,17,8,0.470588,3,0.176471,71.000000
11,6,red,20,14,0.700000,8,0.400000,74.000000
13,7,red,20,11,0.550000,5,0.250000,68.000000
15,8,red,23,13,0.565217,5,0.217391,83.600000
17,9,red,25,11,0.440000,5,0.200000,71.200000
19,10,red,30,16,0.533333,7,0.233333,79.000000


In [ ]:
blue_data.describe().T

,count,mean,std,min,25%,50%,75%,max
day,60.000000,30.500000,17.464249,1.000000,15.750000,30.500000,45.250000,60.000000
color_count,60.000000,25.000000,4.190384,13.000000,21.000000,25.000000,28.000000,34.000000
click_count,60.000000,12.566667,3.702343,7.000000,9.000000,11.500000,15.000000,23.000000
click_rate,60.000000,0.502406,0.111520,0.269231,0.409091,0.500000,0.594444,0.766667
purchase_count,60.000000,7.000000,2.590727,2.000000,5.000000,7.000000,9.000000,15.000000
purchase_rate,60.000000,0.283225,0.099743,0.083333,0.200000,0.270468,0.347115,0.535714
avg_revenue,60.000000,73.459924,4.501617,62.000000,70.200000,73.909091,76.321429,81.500000


In [ ]:
red_data.describe().T

,count,mean,std,min,25%,50%,75%,max
day,60.000000,30.500000,17.464249,1.000000,15.750000,30.500000,45.250000,60.000000
color_count,60.000000,25.000000,4.190384,16.000000,22.000000,25.000000,29.000000,37.000000
click_count,60.000000,13.283333,4.202871,5.000000,10.000000,13.000000,16.000000,26.000000
click_rate,60.000000,0.525298,0.113322,0.217391,0.468326,0.529825,0.601786,0.724138
purchase_count,60.000000,7.133333,2.690011,1.000000,5.000000,7.000000,9.000000,14.000000
purchase_rate,60.000000,0.281965,0.083899,0.041667,0.229895,0.288018,0.333333,0.482759
avg_revenue,60.000000,75.540656,4.716935,64.400000,71.897222,75.428571,78.541667,88.500000


In [ ]:
# Veriyi tarihe göre pivot tablo olarak düzenleyin
pivot_click_count = df.pivot(index='day', columns='color', values='click_count')
pivot_purchase_rate = df.pivot(index='day', columns='color', values='purchase_count')
pivot_click_rate = df.pivot(index='day', columns='color', values='click_rate')
pivot_purchase_rate = df.pivot(index='day', columns='color', values='purchase_rate')
pivot_avg_revenue = df.pivot(index='day', columns='color', values='avg_revenue')

In [ ]:
pivot_click_count.head()

color,blue,red
day,,
1,9,12
2,9,21
3,11,17
4,18,14
5,21,8


In [ ]:
pivot_purchase_rate.head()

color,blue,red
day,,
1,0.240000,0.440000
2,0.277778,0.343750
3,0.350000,0.233333
4,0.535714,0.272727
5,0.303030,0.176471


In [ ]:
pivot_click_rate.head()

color,blue,red
day,,
1,0.360000,0.480000
2,0.500000,0.656250
3,0.550000,0.566667
4,0.642857,0.636364
5,0.636364,0.470588


In [ ]:
pivot_purchase_rate.head()

color,blue,red
day,,
1,0.240000,0.440000
2,0.277778,0.343750
3,0.350000,0.233333
4,0.535714,0.272727
5,0.303030,0.176471


In [ ]:
pivot_avg_revenue.head()

color,blue,red
day,,
1,74.333333,74.818182
2,75.400000,73.181818
3,73.571429,75.428571
4,74.666667,71.833333
5,70.200000,71.000000


### AB TEST GRUP ORTALAMALARI

###### İKİ GRUP FARKLILIKLARINI KARŞILAŞTIRMAK İÇİN BAĞIMSIZ ÖRNEKLEM T TESTİ KULLANILIR.VE BU TEST İÇİN 2 VARSAYIMIN SAĞLANMASI GEREKMEKTEDİR.

###### *1.Varsayım: Normallik Varsayımı*
###### *2.Varsayım Homojenlik Varsayımı.*

In [ ]:
print("df_Blue Click Mean:", blue_data.click_count.mean())
print("df_Red Click Mean:", red_data.click_count.mean())

df_Blue Click Mean: 12.566666666666666
df_Red Click Mean: 13.283333333333333


#### **1.Hipotez Testi: Tıklama Ortalamaları**

###### H0 : M1 = M2  (Kırmızı ekran ve mavi ekranın tıklanma ortalamaları arasında fark yoktur.)
###### H1 : M1!= M2 (Kırmızı ekran ve mavi ekranın tıklanma ortalamaları arasında fark vardır.)

     
##### **1.NORMALLİK VARSAYIMI(SHAPIRO-WILK TESTİ)**
##### H0 : NORMAL DAĞILIM VARSAYIMI SAĞLANMAKTADIR.
##### H1 : NORMAL DAĞILIM VARSAYIMI SAĞLANMAMAKTADIR.

In [ ]:
test_stat, pvalue = shapiro(df.loc[df["color"] == "blue", "click_count"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9319, p-value = 0.0024




```
# p-value=0.0024 < 0.05
# HO reddedilir. Blue grubunun değerleri normal dağılım varsayımını sağlamamaktadır.
```



In [ ]:
test_stat, pvalue = shapiro(df.loc[df["color"] == "red", "click_count"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9780, p-value = 0.3515




```
# p-value=0.9920 > 0.05
# HO reddedilemez. Red grubunun değerleri normal dağılım varsayımını sağlamaktadır.
```



` Burada nonparametrik test kullanacağımız için 2 grup medyanlarını karşılaştırmış oluyoruz. `

In [ ]:
print("df_Blue Click Median:", blue_data.click_count.median())
print("df_Red Click Median:", red_data.click_count.median())

df_Blue Click Median: 11.5
df_Red Click Median: 13.0


In [ ]:
test_stat, pvalue = stats.mannwhitneyu(df.loc[df["color"] == "blue", "click_count"],
                              df.loc[df["color"] == "red", "click_count"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 1614.5000, p-value = 0.3299




```
# P-VALUE=0.3299 > ALFA = 0.05 OLDUGU ICIN H0 REDDEDILEMEZ.
# %95 güvenle Kırmızı ekran ve mavi ekranın tıklanma ortalamaları arasında istatistiksel olarak anlamlı bir fark yoktur.
```



In [ ]:
print("df_Blue Purchase Mean:", blue_data.purchase_count.mean())
print("df_Red Purchase Mean:", red_data.purchase_count.mean())

df_Blue Purchase Mean: 7.0
df_Red Purchase Mean: 7.133333333333334


#### **2.Hipotez Testi: Satınalma Ortalamaları**

###### H0 : M1 = M2  (Kırmızı ekran ve mavi ekranın satınalınma ortalamaları arasında fark yoktur.)
###### H1 : M1!= M2 (Kırmızı ekran ve mavi ekranın satınalınma ortalamaları arasında fark vardır.)

##### **1.NORMALLİK VARSAYIMI(SHAPIRO-WILK TESTİ)**
##### H0 : NORMAL DAĞILIM VARSAYIMI SAĞLANMAKTADIR.
##### H1 : NORMAL DAĞILIM VARSAYIMI SAĞLANMAMAKTADIR.

In [ ]:
test_stat, pvalue = shapiro(df.loc[df["color"] == "blue", "purchase_count"])
print('Blue Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

test_stat, pvalue = shapiro(df.loc[df["color"] == "red", "purchase_count"])
print('Red Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Blue Test Stat = 0.9522, p-value = 0.0197
Red Test Stat = 0.9677, p-value = 0.1122




```
# Blue p-value=0.0197 > 0.05
# HO reddedilemez. Blue grubunun değerleri normal dağılım varsayımını sağlamaktadır.

# Red p-value=0.1122 > 0.05
# HO reddedilemez. Red grubunun değerleri normal dağılım varsayımını sağlamaktadır.
```



##### **2.VARYANS HOMOJENLİĞİ VARSAYIMI (LEVENE TESTİ)**

##### H0: VARYANSLAR HOMOJENDİR.
##### H1: VARYANSLAR HOMOJEN DEĞİLDİR.

In [ ]:
test_stat, pvalue = levene(df.loc[df["color"] == "blue", "purchase_count"],
                           df.loc[df["color"] == "red", "purchase_count"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.0480, p-value = 0.8269




```
# P-VALUE=0.8269 > ALFA = 0.05 OLDUGU ICIN H0 REDDEDILEMEZ.
# Varyans Homojenliği sağlandı. Bağımsız 2 örneklem t testi yapacağız.
```



In [ ]:
test_stat, pvalue = ttest_ind(df.loc[df["color"] == "blue", "purchase_count"],
                              df.loc[df["color"] == "red", "purchase_count"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.2765, p-value = 0.7826




```
# P-VALUE=0.7826 > ALFA = 0.05 OLDUGU ICIN H0 REDDEDILEMEZ.
# %95 güvenle Kırmızı ekran ve mavi ekranından elde edilen satınalma ortalamaları arasında istatistiksel olarak anlamlı bir fark yoktur.
```



In [ ]:
print("df_Blue Revenue Mean:", blue_data.avg_revenue.mean())
print("df_Red Revenue Mean:", red_data.avg_revenue.mean())

df_Blue Revenue Mean: 73.45992405742408
df_Red Revenue Mean: 75.54065577940578


#### **3.Hipotez Testi: Gelir Ortalamaları**

###### H0 : M1 = M2  (Kırmızı ekran ve mavi ekrandan elde edilen gelir ortalamaları arasında fark yoktur.)
###### H1 : M1!= M2 (Kırmızı ekran ve mavi ekrandan elde edilen gelir ortalamaları arasında fark vardır.)

##### **1.NORMALLİK VARSAYIMI(SHAPIRO-WILK TESTİ)**
##### H0 : NORMAL DAĞILIM VARSAYIMI SAĞLANMAKTADIR.
##### H1 : NORMAL DAĞILIM VARSAYIMI SAĞLANMAMAKTADIR.

In [ ]:
test_stat, pvalue = shapiro(df.loc[df["color"] == "blue", "avg_revenue"])
print('Blue Group Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

test_stat, pvalue = shapiro(df.loc[df["color"] == "red", "avg_revenue"])
print('Red Group Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Blue Group Test Stat = 0.9657, p-value = 0.0892
Red Group Test Stat = 0.9799, p-value = 0.4262


```
# Blue p-value=0.0892 > 0.05
# HO reddedilemez. Blue grubunun değerleri normal dağılım varsayımını sağlamaktadır.

# Red p-value=0.4262 > 0.05
# HO reddedilemez. Red grubunun değerleri normal dağılım varsayımını sağlamaktadır.
```

##### **2.VARYANS HOMOJENLİĞİ VARSAYIMI (LEVENE TESTİ)**

##### H0: VARYANSLAR HOMOJENDİR.
##### H1: VARYANSLAR HOMOJEN DEĞİLDİR.

In [ ]:
test_stat, pvalue = levene(df.loc[df["color"] == "blue", "avg_revenue"],
                           df.loc[df["color"] == "red", "avg_revenue"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.0940, p-value = 0.7597




```
# P-VALUE=0.7597 > ALFA = 0.05 OLDUGU ICIN H0 REDDEDILEMEZ.
# Varyans Homojenliği sağlandı. Bağımsız 2 örneklem t testi yapacağız.
```



In [ ]:
test_stat, pvalue = ttest_ind(df.loc[df["color"] == "blue", "avg_revenue"],
                              df.loc[df["color"] == "red", "avg_revenue"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -2.4719, p-value = 0.0149




```
# P-VALUE=0.0149 < ALFA = 0.05 OLDUGU ICIN H0 RED EDİLİR.
# %95 güvenle Kırmızı ekran ve mavi ekrandan elde edilen gelir ortalamaları arasında istatistiksel olarak anlamlı bir fark vardır.
```



### AB TEST ORAN

##### İKİ GRUP ORANLARI ARASINDA KARŞILAŞTIRMA YAPMAK İÇİN KULLANILIR. VARSAYIM YOKTUR.

##### **1.Hipotez Testi: Purchase/Click**

###### Tıklama yapanların getirdiği satın alma oranları.Tıklanma başına satın alma oranı

###### H0 : M1 = M2  (Kırmızı ekran ve mavi ekrana tıklamalardan gelen satın alma oranları arasında fark yoktur.)
###### H1 : M1!= M2 (Kırmızı ekran ve mavi ekrana tıklamalardan gelen satın alma oranları arasında fark vardır.)

In [ ]:
blue_data_purchase_click_rate = (blue_data.purchase_count.sum() / blue_data.click_count.sum()) * 100
print('Blue Group Purchase Click Rate:', blue_data_purchase_click_rate)

red_data_purchase_click_rate = (red_data.purchase_count.sum() / red_data.click_count.sum()) * 100
print('Red Group Purchase Click Rate:', red_data_purchase_click_rate)

Blue Group Purchase Click Rate: 55.702917771883286
Red Group Purchase Click Rate: 53.70138017565872


In [ ]:
test_stat, pvalue = proportions_ztest(count=[blue_data.purchase_count.sum(), red_data.purchase_count.sum()],
                                      nobs=[blue_data.click_count.sum(), red_data.click_count.sum()])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.7914, p-value = 0.4287




```
# P-VALUE=0.4287 > ALFA = 0.05 OLDUGU ICIN H0 REDDEDİLEMEZ.
# Kırmızı ekran ve mavi ekranına yapılan tıklamalardan elde edilen satınalma oranları arasında istatistiksel olarak anlamlı bir fark yoktur.
```



##### **2.Hipotez Testi: Avg. Revenue / Click**

###### Tıklama yapanların getirdiği gelir oranları.Tıklanma başına gelir oranı

###### H0 : M1 = M2  (Kırmızı ekran ve mavi ekrana tıklamalardan gelen ortalama kazanç oranları arasında fark yoktur.)
###### H1 : M1!= M2 (Kırmızı ekran ve mavi ekrana tıklamalardan gelen ortalama kazanç oranları arasında fark vardır.)

In [ ]:
blue_data_purchase_click_rate = (blue_data.avg_revenue.count() / blue_data.click_count.sum()) * 100
print('Blue Group Avg.Revenue Click Rate:', blue_data_purchase_click_rate)

red_data_purchase_click_rate = (red_data.avg_revenue.count() / red_data.click_count.sum()) * 100
print('Red Group Avg.Revenue Click Rate:', red_data_purchase_click_rate)

Blue Group Avg.Revenue Click Rate: 7.957559681697612
Red Group Avg.Revenue Click Rate: 7.528230865746549


In [ ]:
test_stat, pvalue = proportions_ztest(count=[blue_data.avg_revenue.count(), red_data.avg_revenue.count()],
                                      nobs=[blue_data.click_count.sum(), red_data.click_count.sum()])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.3163, p-value = 0.7518


```
# P-VALUE=0.7518 > ALFA = 0.05 OLDUGU ICIN H0 REDDEDİLEMEZ.
# Kırmızı ekran ve mavi ekrana tıklamalardan gelen ortalama kazanç oranları arasında istatistiksel olarak anlamlı bir fark yoktur.
```

### 1.NOT:
Eğer Normallik Varsayımı Sağlansa ve Varyans Homojenliği Varsayımı sağlanmasaydı o zaman Welch Testi uygulayacaktır. Welch Testi Uygulaması
test_stat, pvalue = ttest_ind(df.loc[df["color"] == "blue", "avg_revenue"],
                              df.loc[df["color"] == "red", "avg_revenue"],
                              equal_var=True) # equal_var=False dediğimizde Welch testi olur.

### 2.NOT:
Case Örneği için: Conversion Rate: Purchase/Impression ve Click Through Rate: Click/Impression karşılaştırması yapabilirsiniz. Coversion Rate ve Click Through Rate. Pazarlamada yer alan KPI'lardır.